In [3]:
from google import genai
import os
import time
from IPython.display import Markdown
import subprocess
import sys

In [14]:
client = genai.Client(api_key=os.environ.get('GEMINI_API_KEY'))

print('My files:')
for f in client.files.list():
    print(" ", f'{f.name}: {f.uri}')
    client.files.delete(name=f.name)


My files:


In [15]:
import cv2

def resize_video(input_path, output_path, width=None, height=None):
    # Open the video file
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: Could not open video file {input_path}")
        return
    
    # Get original dimensions and frame rate
    original_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    print(f"Original video size: {original_width}x{original_height}, FPS: {fps}")
    
    # Determine new size while maintaining aspect ratio if needed
    if width is None and height is None:
        new_width, new_height = original_width, original_height
    elif width is None:
        new_width = int(original_width * (height / original_height))
        new_height = height
    elif height is None:
        new_height = int(original_height * (width / original_width))
        new_width = width
    else:
        new_width, new_height = width, height
    
    print(f"Resized video size: {new_width}x{new_height}")
    
    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for MP4
    out = cv2.VideoWriter(output_path, fourcc, fps, (new_width, new_height))
    
    # Process each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Resize the frame
        resized_frame = cv2.resize(frame, (new_width, new_height), interpolation=cv2.INTER_AREA)
        
        # Write the resized frame
        out.write(resized_frame)
    
    # Release resources
    cap.release()
    out.release()
    print(f"Video resized successfully. Saved as '{output_path}'.")
        

resize_video("PostsOfCats-1888852858608521343-01.mp4", "resized.mp4", width=None, height=480)

Original video size: 576x1024, FPS: 30
Resized video size: 270x480
Video resized successfully. Saved as 'resized.mp4'.


In [4]:
client = genai.Client(api_key=os.environ.get('GEMINI_API_KEY'))

print("Uploading file...")
video_file = client.files.upload(file="videos/ujythgrfe.mp4")
print(f"Completed upload: {video_file.uri}")


# Check whether the file is ready to be used.
while video_file.state.name == "PROCESSING":
    print('.', end='')
    time.sleep(1)
    video_file = client.files.get(name=video_file.name)

if video_file.state.name == "FAILED":
    raise ValueError(video_file.state.name)

print('Done')
print('Video name:', video_file.name)

Uploading file...
Completed upload: https://generativelanguage.googleapis.com/v1beta/files/rlvddlre4f1e
..Done
Video name: files/rlvddlre4f1e


In [17]:
video_file_name = client.files.get(name=video_file.name)

# Pass the video file reference like any other media part.
response = client.models.generate_content(
    model="gemini-1.5-pro",
    contents=[
        video_file,
        "Summarize this video. Then create a quiz with answer key "
        "based on the information in the video."])

In [20]:
print(response.candidates[0].content.parts[0].text)

This video shows a tabby cat sitting on a computer tower. The cat appears to be talking or meowing at the camera.

**Quiz Questions:**

1. **What kind of animal is featured in the video?**
   a) Dog
   b) Cat
   c) Hamster
   d) Bird

2. **Where is the animal sitting?**
   a) On a desk
   b) On a chair
   c) On a computer tower
   d) On a laptop

3. **What color is the computer tower?**
   a) Black
   b) White
   c) Gray
   d) Blue

4. **What is the cat doing in the video?**
   a) Sleeping
   b) Eating
   c) Playing
   d) Meowing/Talking

5. **What color is the light emanating from inside the computer tower?**
    a) Red
    b) Green
    c) Blue
    d) Yellow



**Answer Key:**

1. b
2. c
3. b
4. d
5. c


In [ ]:
client.files.delete(name=video_file_name)

In [1]:
import requests 
import ffmpeg
import sys

in_filename = "videos/PostsOfCats-1888852858608521343-01.mp4"
out_filename = "videos/THUMBNAIL.jpg"

def generate_thumbnail(in_filename, out_filename):
    probe = ffmpeg.probe(in_filename)
    time = float(probe['streams'][0]['duration']) // 2
    width = probe['streams'][0]['width']
    try:
        (
            ffmpeg
            .input(in_filename, ss=time)
            .filter('scale', width, -1)
            .output(out_filename, vframes=1)
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )
    except ffmpeg.Error as e:
        print(e.stderr.decode(), file=sys.stderr)
        sys.exit(1)

generate_thumbnail(in_filename, out_filename)

FileNotFoundError: [WinError 2] Le fichier spécifié est introuvable